<a href="https://colab.research.google.com/github/DenisKanzeparov/Transformation/blob/main/Transformation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
from sqlalchemy import create_engine, text

In [2]:
db_config = {
'user': 'praktikum_student', # имя пользователя,
'pwd': 'Sdf4$2;d-d30pp', # пароль,
'host': 'rc1b-wcoijxj3yxfsf3fs.mdb.yandexcloud.net',
'port': 6432, # порт подключения,
'db': 'data-science-vehicle-db' # название базы данных,
}

#сторока подключения к базе данных
connection_string = 'postgresql://{}:{}@{}:{}/{}'.format(
    db_config['user'],
    db_config['pwd'],
    db_config['host'],
    db_config['port'],
    db_config['db'],
)

engine = create_engine(connection_string)

In [3]:
sql_query = """
SELECT *
FROM parties p
WHERE p.party_type IN ('road signs', 'car', 'road bumper', 'building')
LIMIT 100
"""

In [4]:
with engine.connect() as connection:
    df = pd.read_sql_query(text(sql_query), con=connection)

In [5]:
df

,id,case_id,party_number,party_type,at_fault,insurance_premium,party_sobriety,party_drug_physical,cellphone_in_use
0,22,3899454,1,road signs,1,29.0,had not been drinking,None,0.0
1,23,3899454,2,road signs,0,7.0,had not been drinking,None,0.0
2,29,3899462,2,car,0,21.0,had not been drinking,None,0.0
3,31,3899465,2,road signs,0,24.0,had not been drinking,None,0.0
4,41,3899478,2,road bumper,0,NaN,not applicable,not applicable,0.0
...,...,...,...,...,...,...,...,...,...
95,935,4000508,2,road bumper,0,NaN,not applicable,not applicable,NaN
96,948,4000516,3,road bumper,0,NaN,not applicable,not applicable,NaN
97,952,4000518,2,road bumper,0,NaN,had not been drinking,None,0.0
98,978,4000533,1,car,1,NaN,impairment unknown,G,0.0


In [6]:
def clean(string):
  cleaned_str = string.replace(' ', '_')
  return cleaned_str

In [7]:
df['party_type'] = df['party_type'].apply(clean)
df.head(10)

,id,case_id,party_number,party_type,at_fault,insurance_premium,party_sobriety,party_drug_physical,cellphone_in_use
0,22,3899454,1,road_signs,1,29.0,had not been drinking,None,0.0
1,23,3899454,2,road_signs,0,7.0,had not been drinking,None,0.0
2,29,3899462,2,car,0,21.0,had not been drinking,None,0.0
3,31,3899465,2,road_signs,0,24.0,had not been drinking,None,0.0
4,41,3899478,2,road_bumper,0,NaN,not applicable,not applicable,0.0
5,43,3899481,2,road_bumper,0,NaN,not applicable,not applicable,0.0
6,72,3899505,2,road_bumper,0,NaN,not applicable,not applicable,0.0
7,73,3899505,3,road_bumper,0,NaN,not applicable,not applicable,0.0
8,76,3899509,1,road_bumper,0,NaN,had not been drinking,None,0.0
9,93,3981292,2,road_bumper,0,NaN,had not been drinking,None,NaN


In [8]:
dict = {'road_signs': 'Отбойник', 'road_bumper': 'Дорожные знаки', 'car': 'Авто', 'building': 'Строения'}
df['party_type'] = df['party_type'].replace(dict)
df

,id,case_id,party_number,party_type,at_fault,insurance_premium,party_sobriety,party_drug_physical,cellphone_in_use
0,22,3899454,1,Отбойник,1,29.0,had not been drinking,None,0.0
1,23,3899454,2,Отбойник,0,7.0,had not been drinking,None,0.0
2,29,3899462,2,Авто,0,21.0,had not been drinking,None,0.0
3,31,3899465,2,Отбойник,0,24.0,had not been drinking,None,0.0
4,41,3899478,2,Дорожные знаки,0,NaN,not applicable,not applicable,0.0
...,...,...,...,...,...,...,...,...,...
95,935,4000508,2,Дорожные знаки,0,NaN,not applicable,not applicable,NaN
96,948,4000516,3,Дорожные знаки,0,NaN,not applicable,not applicable,NaN
97,952,4000518,2,Дорожные знаки,0,NaN,had not been drinking,None,0.0
98,978,4000533,1,Авто,1,NaN,impairment unknown,G,0.0


In [9]:
pivot_table = df.groupby('party_type').agg({'id': 'count', 'insurance_premium': 'sum'}).reset_index()
pivot_table = pivot_table.rename(columns={'id': 'Количество', 'insurance_premium': 'Сумма страховки (тыс. $)', 'party_type': 'Тип участника происшествия'})
pivot_table

,Тип участника происшествия,Количество,Сумма страховки (тыс. $)
0,Авто,10,380.0
1,Дорожные знаки,65,0.0
2,Отбойник,18,644.0
3,Строения,7,312.0


In [10]:
pivot_table = pivot_table.pivot_table(index=None, columns='Тип участника происшествия', values=['Количество', 'Сумма страховки (тыс. $)']).fillna(0)
pivot_table

Тип участника происшествия,Авто,Дорожные знаки,Отбойник,Строения
Количество,10.0,65.0,18.0,7.0
Сумма страховки (тыс. $),380.0,0.0,644.0,312.0


In [11]:
pivot_table.loc['Количество']

Тип участника происшествия
Авто              10.0
Дорожные знаки    65.0
Отбойник          18.0
Строения           7.0
Name: Количество, dtype: float64

In [12]:
pivot_table.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2 entries, Количество to Сумма страховки (тыс. $)
Data columns (total 4 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Авто            2 non-null      float64
 1   Дорожные знаки  2 non-null      float64
 2   Отбойник        2 non-null      float64
 3   Строения        2 non-null      float64
dtypes: float64(4)
memory usage: 188.0+ bytes
